# Fetch Data
In this notebook we extract all the liks 0f first 250 page of <a href="https://www.truecar.com" >Truecar</a> website. the number of pages can be changed with MAX_PAGE variable.
after the page's extraction, we have to extract all the urls of the car ads so we can get the information of each car.

importing the needed modules

In [4]:
from bs4 import BeautifulSoup
import requests
import re
import csv

In [11]:
MAX_PAGE  = 250
# defining base_url to create a list of all page urls
BASE_URL  = "https://www.truecar.com/used-cars-for-sale/listings/"
# defining host_name for creating urls for each car ad
HOST_NAME = "https://www.truecar.com" 

in this part we are going to create a list of all page urls and loop throuhg all pages to extract all urls of the cars.<br>

output files :<br>

* **pages.txt** : containing all the page urls<br>
* **urls.txt** : containing all the urls<br>
> Pages algorithm: https://www.truecar.com/used-cars-for-sale/listings/?page=num

In [10]:
pages = list()
urls = list()
failed_pages = list()

def url_scraper():
    with open("pages.txt", "w") as f:
        for i in range(1,MAX_PAGE+1):
            page = BASE_URL + "?page=" + str(i)
            f.write(page+"\n")
            pages.append(page)

    for page in pages:
        try:
            response = requests.get(page)
            response.raise_for_status()
        except:
            failed_pages.append(page)
            continue

        src = response.text
        soup = BeautifulSoup(src, "html.parser")

        ads = soup.find_all("a", attrs={
            "data-test" : "vehicleCardLink"
        })
         
        url_list = [HOST_NAME+link["href"] for link in ads]
        with open("urls.txt", "a+") as f:
            for url in url_list:
                urls.append(url)
                f.write(url+"\n")

The main scraping part happens in this part. we loop through all car urls and try to extract its information.
<br>
> **Notice** : this is a long process. make sure you have a stable internet connection and dont forget to run the next block of code before closing notebook to save extracted data.

In [ ]:
car_list = ()
failed_urls = list()

def scraper(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except:
        failed_urls.append(url)
        pass

        try:
            src = response.text
            soup = BeautifulSoup(src, "html.parser")

            detail = soup.find("div", attrs={"data-test" : "vdpPreProspectTopDetails"})

            price = detail.find("div", attrs={"data-test" : "vdpPreProspectPrice"})
            price = int(re.sub("[^\d]", "", price.text))

            mileage = detail.find("p", attrs={"class" : "margin-top-1"})
            mileage = int(re.sub("[^\d]", "", mileage.text))

            titles = soup.find("h1", attrs={
                "class" : "heading-base d-flex flex-column margin-right-2",
                "data-qa" : "Heading"
            })
            name = list(titles.children)[1].text
            name = name.replace("\xa0"," ")

            features = list(soup.find("div", attrs={"data-test" : "vdpOverviewSection"}).div)
            style = features[0].div.div.p.text
            exterior_color = features[1].div.div.p.text
            interior_color = features[2].div.div.p.text
            mpg = features[3].div.div.p.text
            mpg_city, mpg_highway = re.match("(\d{1,2}) cty / (\d{1,2}) hwy", mpg).groups()
            engine = features[4].div.div.p.text
            drive_type = features[5].div.div.p.text
            fuel_type = features[6].div.div.p.text
            transmission = features[7].div.div.p.text

        except:
            failed_urls.append(url)
            pass
            
 
        car = [name,style,exterior_color,interior_color,engine,
                    drive_type,fuel_type,transmission,mileage,mpg_city,
                    mpg_highway,price]

        car_list.append(car)        

exporting a csv file

In [ ]:
with open("cars.csv", "w") as cars:
    csvwriter = csv.writer(cars)
    csvwriter.writerows(car_list)

Sina Kazemi<br>
Github : <a href="https://github.com/sina96n/">sina96n</a>